In [1]:
import pymongo as pym
import nltk.data
import re
import string
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import TreebankWordTokenizer
import stop_words
from nltk.stem import *
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from scipy.sparse import hstack
from sklearn.linear_model import LogisticRegression

In [2]:
print(stopwords.words('french'))
print('---')
print(stop_words.get_stop_words('fr'))

['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'je', 'la', 'le', 'leur', 'lui', 'ma', 'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne', 'nos', 'notre', 'nous', 'on', 'ou', 'par', 'pas', 'pour', 'qu', 'que', 'qui', 'sa', 'se', 'ses', 'son', 'sur', 'ta', 'te', 'tes', 'toi', 'ton', 'tu', 'un', 'une', 'vos', 'votre', 'vous', 'c', 'd', 'j', 'l', 'à', 'm', 'n', 's', 't', 'y', 'été', 'étée', 'étées', 'étés', 'étant', 'étante', 'étants', 'étantes', 'suis', 'es', 'est', 'sommes', 'êtes', 'sont', 'serai', 'seras', 'sera', 'serons', 'serez', 'seront', 'serais', 'serait', 'serions', 'seriez', 'seraient', 'étais', 'était', 'étions', 'étiez', 'étaient', 'fus', 'fut', 'fûmes', 'fûtes', 'furent', 'sois', 'soit', 'soyons', 'soyez', 'soient', 'fusse', 'fusses', 'fût', 'fussions', 'fussiez', 'fussent', 'ayant', 'ayante', 'ayantes', 'ayants', 'eu', 'eue', 'eues', 'eus', 'ai', 'as', 'avons', 'avez', 'ont', 'aurai', 'auras', 'aura', 'aurons', 'aurez', 'auront',

In [3]:
stops = set(['rt','ds','qd','ss','ns','vs','nn','amp','gt','gd','gds','tt','pr','ac','mm', 'qu',
            '``', 'ni', 'ca', 'le', 'les', ' ', 'si', '$', '^', 'via', 'ils'] +
            list('@ن%£€‘:&;') + list('abcdefghijklmnopqrstuvwxyz'))
print(stops)

{'‘', 'mm', 'vs', 'gds', 'si', 'b', 'qd', 'q', 'r', '``', 'e', 'gd', 'k', 'm', 'ن', 's', 'd', 'u', 'qu', '&', 'g', 'rt', 'ca', 'gt', ';', '@', 'ss', 'h', 'ac', 'nn', 'ns', 'ni', 'p', 'ds', '%', '$', 'l', 'pr', 'w', 'tt', 'le', 'via', 'a', 'c', 'ils', 'i', 'y', 'o', 'z', 'n', 'j', '£', ':', ' ', 'f', '^', 'x', 'amp', 'v', '€', 'les', 't'}


In [10]:
def tweetPreprocessing(retweet=False):
    client = pym.MongoClient('localhost', 27017)
    #collection = client.tweet.cleaned
    collection = client.tweet.train

    # attention : dans la base "labelised" le champ s'appelle t_text et pas text
    #collection2 = client.tweet.labelised

    tweets = collection.find(filter={'text':{'$exists':True}}, projection={'_id':False})
    #tweets2 = collection2.find(filter={'t_text':{'$exists':True}}, projection={'_id':False})
    
    df = pd.DataFrame()
    listTweets, listCandidats, listSentiments = [], [], []

    for t in tweets: 
        if not retweet: # filtrage des retweets
            if 'rt @' in t['text']:
                continue
        
        # comptes
        a = t['text'].count('!')
        b = t['text'].count('?')
        c = t['text'].count('#')
        d = t['text'].count('"')
        e = t['text'].count('http')
#         if e > 1:
#             e = 1
        
        # mot tronqué
        t['text'] = re.sub(r'\w*…', '', t['text'])
        
        # caracteres speciaux, url et rt
        t['text'] = re.sub(r'\xad', '-',
                           re.sub(r'\n', ' ',
                                  re.sub(r'\W*(?!\S)', '',
                                         re.sub(r'(?:htt)\S*', '',
                                                re.sub(r'^rt.*: ', '',
                                                       re.sub(r'\d', '',
                                                              re.sub(r',;:!?\.\/\*(){}', '',
                                                                     re.sub(r'«»', '', t['text']))))))))
        
        t['text'] = re.sub('|'.join(['’', '_', '-', '\'', '\.', '/', '“', '  ']), ' ', t['text'])
        
        # accents
#         t['text'] = re.sub('|'.join('Ééèêë'), 'e', t['text'])
#         t['text'] = re.sub('|'.join('àâä'), 'a', t['text'])
#         t['text'] = re.sub('|'.join('ç'), 'c', t['text'])
#         t['text'] = re.sub('|'.join('œ'), 'oe', t['text'])
#         t['text'] = re.sub('|'.join('Ôôö'), 'o', t['text'])
#         t['text'] = re.sub('|'.join('îï'), 'i', t['text'])
#         t['text'] = re.sub('|'.join('ùû'), 'u', t['text'])
        
        # apostrophes
        t['text'] = re.sub('|'.join([elem + '\'' for elem in 'cdjlmnst']), '', t['text'])
        
        tokenizer = TreebankWordTokenizer()
        t['text'] = tokenizer.tokenize(t['text'])
        t['text'] = [token for token in t['text'] if (token not in stops) and (len(token)>2)]

        while '' in t['text']:
            t['text'].pop('')
            
        if t['text']: # test si liste non vide
            #listTweets.append(list(set(t['text']))) # mots uniques
            listTweets.append(t['text'])
            try:
                listCandidats.append(t['candidat'])
            except:
                listCandidats.append(None)

            try:
                listSentiments.append(t['sentiment'])
            except:
                listSentiments.append(None)
                
            rec = pd.DataFrame([[a, b, c, d, e]], columns=['!', '?', '#', '"', '_http_'])
            df = df.append(rec, ignore_index=True)
        
    df['text'], df['candidat'], df['sentiment'] = listTweets, listCandidats, listSentiments
    
    return df

def build_feat_mat(df_tweets):
    vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', decode_error='strict',
                                use_idf=True, norm='l2', binary=False, min_df=.0005, max_df=1.)
    X = vectorizer.fit_transform(df_tweets['text'].apply(' '.join))
    hstack((X, df_tweets[['!', '?', '#', '"', '_http_']]))

    return X

def getSentiments(n_predict, retweet) : 
    df = tweetPreprocessing(retweet)
    df = df.sample(frac=1.0, replace=False) # mélange des lignes
    
    X = build_feat_mat(df)
    y = df['sentiment']
    
    n_samples, vocabulaire = X.shape
    print('Tweets : ' + str(n_samples) + ' / ' + 'Mots : ' + str(vocabulaire))
    
    #model = MultinomialNB()
    #model = RandomForestClassifier(n_estimators=30, criterion='gini', max_depth=None, n_jobs=-1)
    #model = KNeighborsClassifier(n_neighbors=15, weights='distance', algorithm='auto', leaf_size=30, p=2, metric='minkowski', n_jobs=-1)
    model = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, class_weight='balanced')
    
    model.fit(X[:-n_predict], y[:-n_predict])
    predictions = model.predict(X[n_samples - n_predict:])
    
    print('Score', np.sum(predictions == y[n_samples - n_predict:]) / len(predictions))
    return predictions


In [5]:
corpus = tweetPreprocessing(retweet=True)
print('Taille base d\'entrainement :', corpus.shape[0])
print(corpus['sentiment'].value_counts())

Taille base d'entrainement : 3728
-1.0    2380
 0.0     950
 1.0     398
Name: sentiment, dtype: int64


In [6]:
corpus[:10]

,!,?,#,"""",_http_,text,candidat,sentiment
0,0,0,0,0,1,"[des, français, veulent, interdire, aux, parle...",None,0.0
1,0,0,0,0,1,"[pauvre, fillon, finir, par, devenir, informat...",None,-1.0
2,0,0,1,0,0,"[pas, révolution, simplement, vielles, recette...",None,-1.0
3,0,0,0,0,0,"[cette, presse, étrangère, est, aux, mains, de...",None,-1.0
4,0,0,0,2,1,"[est, comme, avait, pas, pilote, dans, avion]",None,-1.0
5,0,0,0,0,0,"[poireau, très, présent, mais, suis, très, trè...",None,-1.0
6,0,1,0,0,2,"[deux, lois, benoît, hamon, bercy, ont, elles,...",None,0.0
7,0,0,0,1,0,"[bayrou, tweetait, derriere, macron, des, inte...",None,-1.0
8,0,0,0,2,0,"[gifles, valls, frôles, chaise, électrique, vi...",None,-1.0
9,0,0,3,0,1,"[macron, presidentielle, ‼️il, faut, voter, fi...",None,1.0


In [11]:
a = getSentiments(200, retweet=True)
print(len(a[a==1]), len(a[a==0]))

Tweets : 3728 / Mots : 3360
Score 0.675
5 37


In [8]:
# calculer des f-score, sentiwordnet, bigrammes...